# Assocation Rule Mining - FP Growth

In [18]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# Load Dataset

df = pd.read_csv('https://raw.githubusercontent.com/renatomaaliw3/public_files/refs/heads/master/Data%20Sets/software_developer.csv')
df

,Age,Years_of_Experience,Degree,Coding_Test_Score,Interview_Score,Portfolio_Project_Count,Fullstack_Knowledge,Preferred_WE,Certifications,Previous_Job_Level,Leadership_Experience,Hackathon_Participation,Attitude_Evaluation,Status
0,41-50,3-5 years,No Degree,Code_Medium,Interview_Medium,Many Projects,Full_No,On-site,GCP,Prev_Lead,Lead_Yes,Hack_No,ATT_Positive,STATUS_Hired
1,20-30,3-5 years,Master's,Code_Medium,Interview_Low,No Projects,Full_No,Hybrid,GCP,Prev_Mid,Lead_Yes,Hack_No,ATT_Neutral,STATUS_Hired
2,20-30,10+ years,Bachelor's,Code_Medium,Interview_Medium,Few Projects,Full_Yes,On-site,AWS,Prev_Lead,Lead_Yes,Hack_Yes,ATT_Positive,STATUS_Hired
3,20-30,10+ years,No Degree,Code_Medium,Interview_High,Few Projects,Full_No,On-site,Azure,Prev_Mid,Lead_Yes,Hack_Yes,ATT_Positive,STATUS_Hired
4,41-50,10+ years,PhD,Code_Medium,Interview_Low,Few Projects,Full_Yes,Hybrid,Azure,Prev_Mid,Lead_No,Hack_No,ATT_Positive,STATUS_Hired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,41-50,10+ years,No Degree,Code_High,Interview_Medium,Few Projects,Full_Yes,Remote,GCP,Prev_Junior,Lead_Yes,Hack_Yes,ATT_Positive,STATUS_Not_Hired
996,41-50,3-5 years,PhD,Code_High,Interview_Medium,Few Projects,Full_No,On-site,GCP,Prev_Junior,Lead_Yes,Hack_No,ATT_Negative,STATUS_Not_Hired
997,31-40,0-2 years,PhD,Code_High,Interview_Medium,Many Projects,Full_No,On-site,AWS,Prev_Lead,Lead_Yes,Hack_Yes,ATT_Neutral,STATUS_Not_Hired
998,41-50,0-2 years,Master's,Code_Low,Interview_Medium,Few Projects,Full_Yes,Remote,No Certifications,Prev_Junior,Lead_Yes,Hack_Yes,ATT_Neutral,STATUS_Not_Hired


In [20]:
# Data Preprocessing
# Before Applying the FPGrowth Algorithm, we need to preprocess the data
# One-Hot Encoding, Remember get dummies?

from mlxtend.preprocessing import TransactionEncoder

# Consolidate each transaction into a single list of items, removing NaN values
transactions = df.apply(lambda row: row.dropna().tolist(), axis = 1).tolist()

# Initialize TransactionEncoder
encoder = TransactionEncoder()

# Fit and transform the transactions data
transaction_matrix = encoder.fit_transform(transactions)

# Convert to DataFrame
transaction_df = pd.DataFrame(transaction_matrix, columns = encoder.columns_)
transaction_df

# Hired Employees Only

# status_employees = transaction_df.drop('STATUS_Not_Hired', axis = 1)
# status_employees = status_employees[status_employees['STATUS_Hired'] == True]
# status_employees = status_employees.loc[:, :'Remote']
# status_employees

# Not Hired Employees Only

status_employees = transaction_df.drop('STATUS_Hired', axis = 1)
status_employees = status_employees[status_employees['STATUS_Not_Hired'] == True]
status_employees = status_employees.loc[:, :'Remote']
status_employees

,0-2 years,10+ years,20-30,3-5 years,31-40,41-50,6-10 years,ATT_Negative,ATT_Neutral,ATT_Positive,...,No Certifications,No Degree,No Projects,On-site,PhD,Prev_Junior,Prev_Lead,Prev_Mid,Prev_Senior,Remote
500,False,False,False,True,False,True,False,True,False,False,...,False,True,False,False,False,False,False,False,True,False
501,True,False,False,False,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
502,False,True,False,False,False,True,False,True,False,False,...,False,True,False,False,False,False,False,False,True,False
503,False,True,False,False,False,True,False,True,False,False,...,False,True,False,True,False,False,False,False,True,False
504,False,True,True,False,False,False,False,False,False,True,...,True,False,False,False,True,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,True,False,False,False,True,False,False,False,True,...,False,True,False,False,False,True,False,False,False,True
996,False,False,False,True,False,True,False,True,False,False,...,False,False,False,True,True,True,False,False,False,False
997,True,False,False,False,True,False,False,False,True,False,...,False,False,False,True,True,False,True,False,False,False
998,True,False,False,False,False,True,False,False,True,False,...,True,False,False,False,False,True,False,False,False,True


In [21]:
# Appying the FPGrowth Algorithm
# Since data are cleaned and prepared for frequent itemset

from mlxtend.frequent_patterns import fpgrowth, association_rules

# Apply the FPGrowth Algorithm
frequent_itemsets = fpgrowth(status_employees, min_support = 0.3, use_colnames = True)

# min_support is the minimum support threshold. Itemsets with support greater than or equal to this threshold will be returned.
# use_colnames = True ensures that the item names are used in the output instead of column indices.

In [22]:
# View Frequent Itemsets

print(frequent_itemsets)

    support                      itemsets
0     0.576                    (Hack_Yes)
1     0.532                    (Lead_Yes)
2     0.460            (Interview_Medium)
3     0.456                    (Full_Yes)
4     0.444                   (Code_High)
5     0.428                       (41-50)
6     0.364               (Many Projects)
7     0.362                (ATT_Negative)
8     0.530                (Few Projects)
9     0.386                 (ATT_Neutral)
10    0.380                 (Code_Medium)
11    0.544                     (Full_No)
12    0.468                     (Lead_No)
13    0.378              (Interview_High)
14    0.424                     (Hack_No)
15    0.430                      (Remote)
16    0.314                       (20-30)
17    0.326                  (6-10 years)
18    0.376                 (Prev_Junior)
19    0.306  (Lead_Yes, Interview_Medium)
20    0.310          (Lead_Yes, Full_Yes)
21    0.316     (Code_High, Few Projects)
22    0.308      (Hack_Yes, Few Pr

In [23]:
# Generate Association Rules

rules = association_rules(frequent_itemsets, num_itemsets = len(status_employees), metric = "confidence", min_threshold = 0.3)
rules.loc[:, :'lift']

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(Lead_Yes),(Interview_Medium),0.532,0.460,0.306,0.575188,1.250409
1,(Interview_Medium),(Lead_Yes),0.460,0.532,0.306,0.665217,1.250409
2,(Lead_Yes),(Full_Yes),0.532,0.456,0.310,0.582707,1.277866
3,(Full_Yes),(Lead_Yes),0.456,0.532,0.310,0.679825,1.277866
4,(Code_High),(Few Projects),0.444,0.530,0.316,0.711712,1.342852
5,(Few Projects),(Code_High),0.530,0.444,0.316,0.596226,1.342852
6,(Hack_Yes),(Few Projects),0.576,0.530,0.308,0.534722,1.008910
7,(Few Projects),(Hack_Yes),0.530,0.576,0.308,0.581132,1.008910
8,(Full_No),(Lead_No),0.544,0.468,0.322,0.591912,1.264769
9,(Lead_No),(Full_No),0.468,0.544,0.322,0.688034,1.264769


### Hired

In [24]:
# Positive Attitude Association (High Confidence and Lift):

# Many rules have high confidence (close to or above 0.75) and lift
# values close to 1 or slightly above, which suggests that candidates with certain attributes are
# more likely to have a positive attitude, although the association strength varies.

# Candidates with no full-stack knowledge ("Full_No") have a high likelihood of
# being evaluated as positive (76.3% confidence, lift of 1.023).

# Those with "Many Projects" in their portfolio or a "Medium Interview Score" are also
# associated with a positive attitude, with confidence levels around 80% and lift above 1.

# Candidates who scored "High" on coding tests (confidence of 79%) are also associated with positive attitudes,
# reinforcing the idea that both skills and attitude correlate with each other.

In [25]:
# Leadership and Hackathon Experience:

# Those with "Lead_No" or "Hack_No" are both strongly associated with a positive attitude,
# with confidence values above 74% and lift values slightly over 1. This indicates that candidates
# without formal leadership experience or hackathon participation may still receive a positive attitude evaluation.

# However, those who participated in hackathons ("Hack_Yes") also show a positive attitude association (75% confidence),
# suggesting that hackathon experience is linked with a positive evaluation, regardless of other skill areas.

In [26]:
# Correlations Between Candidate Attributes:

# There are mutual associations between "Hack_Yes" and "Full_No," with a lift of 1.179,
# indicating that candidates with hackathon experience are likely to lack full-stack knowledge, and vice versa.

# This may suggest that candidates who focus on hackathons are not necessarily developing
# full-stack expertise but may still be favorably evaluated on attitude.

In [27]:
# Portfolio and Full-stack Knowledge:

# Candidates with "Few Projects" or "Full_Yes" are associated with positive attitude evaluations,
# though with slightly lower confidence (~70%). This suggests that moderate project experience or full-stack knowledge
# does contribute, albeit less directly, to a positive attitude evaluation.

In [28]:
# Conclusion

# The patterns indicate that a positive attitude is not solely dependent on technical qualifications.
# Both strong performers and average programmers can receive favorable evaluations if they demonstrate
# desirable traits like practical project experience or a balanced skill set without full-stack knowledge.
# The findings also emphasize that participation in hackathons, coding scores, and attitude evaluations
# are positively linked, suggesting that a focus on personal projects and coding ability may predict a
# positive attitude, making candidates attractive to top companies even with average technical qualifications.

### Not Hired

In [29]:
# Moderate Interview Scores and Leadership ("Lead_Yes"):

# Candidates with an "Interview_Medium" score have a high association with "Lead_Yes" status,
# with a confidence of 66.5% and a lift of 1.25. Conversely, having leadership experience ("Lead_Yes")
# increases the likelihood of having a medium interview score.

# This suggests that candidates with leadership experience may not consistently excel in interviews,
# possibly due to a misalignment in expectations for leadership roles versus actual performance in interview settings.

In [30]:
# Full-stack Knowledge and Leadership:

# There is a strong association between "Full_Yes" (full-stack knowledge) and "Lead_Yes" (leadership experience),
# with a high confidence of 67.9% and a lift of 1.28. However, candidates with this combination were not hired.

# This indicates that, for these candidates, full-stack skills combined with leadership experience
# may not have been sufficient for hiring. It’s possible that hiring managers prioritize other factors,
# such as specific technical expertise or a particular attitude, over full-stack skills when leadership is already demonstrated.

In [31]:
# Few Projects and High Coding Scores:

# The rule connecting "Few Projects" with "Code_High" shows a strong association,
# with a lift of 1.34 and confidence of 59.6%. High coding scores do not seem to compensate
# for limited project experience in the hiring decision.

# This suggests that a high coding ability without a robust project portfolio may not appeal
# as strongly to hiring managers, who likely favor practical, project-based experience alongside coding skills.

In [32]:
# Hackathon Experience and Few Projects:

# Candidates with hackathon participation ("Hack_Yes") are associated with having "Few Projects", and vice versa,
# though with a lift near 1 (1.01). This implies that hackathon experience alone, without a broad project portfolio,
# may not be viewed as sufficient, possibly due to the limited depth such experience may provide.

In [33]:
# Lack of Leadership and Full-stack Knowledge:

# For candidates who lack leadership experience ("Lead_No"), there is an association with "Full_No" (no full-stack knowledge),
# with a confidence of 68.8% and a lift of 1.26. These candidates were not hired,
# suggesting that the absence of both leadership and full-stack expertise may be seen as limiting in the hiring process.

# Additionally, "Lead_No" candidates also have a moderate association with hackathon participation ("Hack_Yes"),
# indicating that hackathon experience alone, without leadership or full-stack skills, might not fulfill hiring criteria.

In [34]:
# Conclusion

# Leadership and Technical Skill Misalignment: Candidates with leadership experience may
# need to excel in specific technical evaluations or projects to compensate for perceived gaps in other areas.

# Insufficient Project-Based Experience: High coding skills without a breadth of project
# experience seem less compelling to hiring managers, suggesting the importance of applied,
# project-based evidence of skills.

# Lack of Key Skills and Experience: Candidates lacking both full-stack knowledge and leadership experience
# are at a disadvantage, even with other positive attributes like hackathon participation.